In [ ]:
import sys 
from importlib import reload
import utils
reload(utils)
from utils import *

In [ ]:
agricensus = pd.read_csv('data/raw/Unità agricole e numero di capi per tipo di allevamento e zona altimetrica (IT1,DF_DCAT_CENSAGRIC2020_AU_CATTLE_1,1.0).csv')
comuni_piemonte = pd.read_csv('data/clean_data/elenco_comuni_piemonte.csv')
print(agricensus['DATAFLOW'].value_counts(), '\n')
print(agricensus['HEADS_NUMBER'].value_counts(), '\n')
print(agricensus['FREQ'].value_counts(), '\n')
print(agricensus['TIME_PERIOD'].value_counts(), '\n')

**METADATA**
- *DATAFLOW:* colonna invariante
- *FREQ:* frequenza raccolta dati (A = annuale, colonna invariante)
- *REF_AREA:* codice della granularità del dato (solo numerico = comune. Numero a 6 cifre, anticipato da zeri in caso di difetto)
- *DATA_TYPE:* non pervenuto
- *TYPE_OF_HERD:* tipo di risorsa (alveare, bovina, suina...)
- *HEADS_NUMBER:* categoria altometrica a cui fa capo (colonna invariante TOT)
- *ALTIMETRIC_ZONE:* categoria altimetrica (pianura, collina, alta montagna...) NOTA: l'altometrai è registrata a livello comunale, ergo 1 solo tipo per comune (TOT coincide con il tipo)
- *TIME_PERIOD:* periodo di registrazione del dato in relazione a FREQ (in questo caso, anno 2020 invariante)
- *OBS_VALUE:* quantità della risorsa (numero intero)
- tutte le altre variabili in NAN 

In [ ]:
agricensus.loc[:, 'REF_AREA'] = 'A' + pd.Series([str(code) for code in agricensus.loc[:, 'REF_AREA']]).str.zfill(6)
mask = agricensus['REF_AREA'].isin(comuni_piemonte['Codice Comune'])
print(agricensus[mask].info())

In [ ]:
grazing_census = agricensus[mask].drop(agricensus.columns[9:], axis=1
                ).drop(['DATAFLOW', 'HEADS_NUMBER', 'FREQ', 'TIME_PERIOD'], axis=1)
save_clean_data(grazing_census, 'agricoltura/grazing_census_2020')

---

2010

In [ ]:
agricensus = pd.read_csv('data/raw/agricensus_allevamenti_2010.csv', header=1, encoding='ISO-8859-1', sep=';')
agricensus.info()

In [ ]:
agricensus[agricensus['Territorio'].isna()]

In [ ]:
agricensus.loc[163, 'Territorio'] = 'None Torinese'
agricensus = agricensus.apply(lambda x: uniform_strings(x, 'Territorio'), axis=1)

In [ ]:
agricensus[agricensus['Territorio'] == 'nonetorinese']

In [ ]:
agricensus.describe()

In [ ]:
# remove gathered data of provinces
grazing = agricensus.drop_duplicates(subset='Territorio', keep='last')

In [ ]:
grazing.info()

In [ ]:
check_strange_letters(grazing, 'Territorio')

In [ ]:
grazing_piemonte = grazing.merge(comuni_piemonte[['Comune', 'Codice Comune']].rename({'Comune':'Territorio'}, axis=1), how='inner', on='Territorio')
grazing_piemonte.info()

In [ ]:
check_strange_letters(grazing_piemonte, 'Territorio')

In [ ]:
grazing_piemonte.drop('Territorio', axis=1, inplace=True)
save_clean_data(grazing_piemonte, 'agricoltura/grazing_census_2010')